In [1]:
%cd /content/


[WinError 2] The system cannot find the file specified: '/content/'
d:\misc\Programming\Projects\HeadSwap\HeadSwap-Clone


In [2]:
! nvidia-smi


Wed Feb 14 11:50:58 2024       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 551.23                 Driver Version: 551.23         CUDA Version: 12.4     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                     TCC/WDDM  | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  NVIDIA GeForce GTX 1650      WDDM  |   00000000:01:00.0 Off |                  N/A |
| N/A    0C    P3              9W /   30W |       0MiB /   4096MiB |      0%      Default |
|                                         |                        |                  N/A |
+-----------------------------------------+-----

In [3]:
# ! git clone https://github.com/notAllMight1/HeadSwap-Clone.git
%cd HeadSwap-Clone/process

[WinError 3] The system cannot find the path specified: 'HeadSwap-Clone/process'
d:\misc\Programming\Projects\HeadSwap\HeadSwap-Clone


In [4]:
%cd /content/HeadSwap-Clone/process

/content/HeadSwap-Clone/process


In [5]:
cd /content/HeadSwap-Clone

/content/HeadSwap-Clone


In [ ]:
cd /content/HeadSwap-Clone/process

/content/HeadSwap-Clone/process


In [ ]:
 %run NewScript.py

In [ ]:
! wget https://raw.githubusercontent.com/sicxu/Deep3DFaceRecon_pytorch/master/BFM/similarity_Lm3D_all.mat -P ../pretrained_models/BFM

--2024-02-13 08:55:08--  https://raw.githubusercontent.com/sicxu/Deep3DFaceRecon_pytorch/master/BFM/similarity_Lm3D_all.mat
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 994 [application/octet-stream]
Saving to: ‘../pretrained_models/BFM/similarity_Lm3D_all.mat’

similarity_Lm3D_all 100%[===================>]     994  --.-KB/s    in 0s      

2024-02-13 08:55:09 (56.6 MB/s) - ‘../pretrained_models/BFM/similarity_Lm3D_all.mat’ saved [994/994]



In [ ]:
#@title Setup files downloader
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials
import os

download_with_pydrive = True #@param {type:"boolean"}

class Downloader(object):
    def __init__(self, use_pydrive):
        self.use_pydrive = use_pydrive
        current_directory = os.getcwd()
        self.save_dir = '../pretrained_models'
        os.makedirs(self.save_dir, exist_ok=True)
        if self.use_pydrive:
            self.authenticate()

    def authenticate(self):
        auth.authenticate_user()
        gauth = GoogleAuth()
        gauth.credentials = GoogleCredentials.get_application_default()
        self.drive = GoogleDrive(gauth)

    def download_file(self, file_id, file_name):
        file_dst = f'{self.save_dir}/{file_name}'
        if os.path.exists(file_dst):
            print(f'{file_name} already exists!')
            return
        if self.use_pydrive:
            downloaded = self.drive.CreateFile({'id':file_id})
            downloaded.FetchMetadata(fetch_all=True)
            downloaded.GetContentFile(file_dst)
        else:
            !gdown --id $file_id -O $file_dst

downloader = Downloader(download_with_pydrive)

In [ ]:
#@title Download epoch_20
MODEL_PATHS = {"id": "1BlDBB4dLLrlN3cJhVL4nmrd_g6Jx6uP0", "name": "epoch_20.pth"}

downloader.download_file(file_id=MODEL_PATHS["id"], file_name=MODEL_PATHS["name"])

In [ ]:
MODEL_PATHS = {"id": "1-0xOf6g58OmtKtEWJlU3VlnfRqPN9Uq7", "name": "face.zip"}
downloader.download_file(file_id=MODEL_PATHS["id"], file_name=MODEL_PATHS["name"])
%cd ../pretrained_models
! unzip -x face.zip
! mv face/epoch_00190_iteration_000400000_checkpoint.pt ./

/content/HeadSwap-Clone/pretrained_models
Archive:  face.zip
   creating: face/
  inflating: face/epoch_00190_iteration_000400000_checkpoint.pt  
  inflating: face/latest_checkpoint.txt  


In [ ]:
! ls

BFM			  epoch_00190_iteration_000400000_checkpoint.pt  face	   parsing.pth
Blender-401-00012900.pth  epoch_20.pth					 face.zip  sr_cf.onnx


In [ ]:
! pip install torch torchvision numpy opencv-python onnxruntime-gpu face-alignment==1.1.1

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 192.1/192.1 MB 6.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.0/46.0 kB 7.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.8/86.8 kB 14.0 MB/s eta 0:00:00


In [ ]:
%cd ..

/content/HeadSwap-Clone


In [ ]:
from model.AlignModule.generator import FaceGenerator
from model.BlendModule.generator import Generator as Decoder
from model.AlignModule.config import Params as AlignParams
from model.BlendModule.config import Params as BlendParams
from model.third.faceParsing.model import BiSeNet
import torchvision.transforms.functional as TF
import torch.nn.functional as F
import torch
import cv2
import numpy as np
import pdb
from process.process_func import Process
from process.process_utils import *
import os
import onnxruntime as ort


In [ ]:
class Infer(Process):
    def __init__(self,align_path,blend_path,parsing_path,params_path,bfm_folder):
        Process.__init__(self,params_path,bfm_folder)
        align_params = AlignParams()
        blend_params = BlendParams()
        self.device = 'cpu'
        if torch.cuda.is_available():
            self.device = 'cuda'

        self.parsing = BiSeNet(n_classes=19).to(self.device)

        self.netG = FaceGenerator(align_params).to(self.device)
        self.decoder = Decoder(blend_params).to(self.device)

        self.loadModel(align_path,blend_path,parsing_path)
        self.eval_model(self.netG,self.decoder,self.parsing)


        self.ort_session_sr = ort.InferenceSession('./pretrained_models/sr_cf.onnx', providers=['CPUExecutionProvider'])

    def run(self,src_img_path_list,tgt_img_path_list,save_base,crop_align=False,cat=False):
        os.makedirs(save_base,exist_ok=True)
        i = 0
        for src_img_path,tgt_img_path in zip(src_img_path_list,tgt_img_path_list):
            gen = self.run_single(src_img_path,tgt_img_path,crop_align=crop_align,cat=cat)
            img_name = os.path.splitext(os.path.basename(src_img_path))[0]+'-' + \
                        os.path.splitext(os.path.basename(tgt_img_path))[0]+'.png'
            cv2.imwrite(os.path.join(save_base,img_name),gen)
            print('\rhave done %04d'%i,end='',flush=True)
            i += 1
        print()
    def run_single(self,src_img_path,tgt_img_path,crop_align=False,cat=False):

        tgt_img = cv2.imread(tgt_img_path)
        tgt_align = tgt_img.copy()

        tgt_align,info = self.preprocess_align(tgt_img)
        if tgt_align is None:
            return None

        src_img = cv2.imread(src_img_path)
        src_align = src_img
        if crop_align:
            src_align,_ = self.preprocess_align(src_img,top_scale=0.55)

        src_inp = self.preprocess(src_align)
        tgt_inp = self.preprocess(tgt_align)

        tgt_params = self.get_params(cv2.resize(tgt_align,(256,256)),
                                info['rotated_lmk']/2.0).unsqueeze(0)

        gen = self.forward(src_inp,tgt_inp,tgt_params)

        gen = self.postprocess(gen[0])
        gen = self.run_sr(gen)
        mask = self.mask
        final = gen
        # gen = color_transfer2(tgt_align,gen)

        RotateMatrix = info['im'][:2]
        mask = info['mask'][...,0]

        rotate_gen = cv2.warpAffine(gen, RotateMatrix, (tgt_img.shape[1], tgt_img.shape[0]))
        mask = cv2.warpAffine(mask, RotateMatrix, (tgt_img.shape[1], tgt_img.shape[0])) * 1.0

        # ori_mask = mask.copy()
        kernel2 = cv2.getStructuringElement(cv2.MORPH_RECT,(17, 17))
        # mask = cv2.dilate(mask*1.0,kernel2)
        mask = cv2.erode(mask*1.0,kernel2)
        # mask = cv2.GaussianBlur(mask*255.0, (21, 21), 0) / 255.0
        mask = cv2.blur(mask*1.0, (15, 15), 0) / 255.0
        mask = np.clip(mask,0,1.0)[:,:,np.newaxis]

        # pdb.set_trace()
        final = rotate_gen * mask + tgt_img * (1-mask)

        if cat:
            final = np.concatenate([tgt_img,final],1)
            final[-256:,:256] = cv2.resize(src_align,(256,256))

        return final

    def forward(self,xs,xt,params):
        with torch.no_grad():

            # xg = self.netG(F.adaptive_avg_pool2d(xs,256),
            #                 F.adaptive_avg_pool2d(xt,256),
            #                 params)['fake_image']
            xg = F.adaptive_avg_pool2d(self.netG(F.adaptive_avg_pool2d(xs,256),
                            F.adaptive_avg_pool2d(xt,256),
                            params)['fake_image'],512)


            M_a = self.parsing(self.preprocess_parsing(xg))

            M_t = self.parsing(self.preprocess_parsing(xt))

            M_a = self.postprocess_parsing(M_a)
            M_t = self.postprocess_parsing(M_t)
            # xg[M_a.repeat(1,3,1,1)==0] = -0.5
            # xg[M_a.repeat(1,3,1,1)==16] = 0.6
            xg_gray = TF.rgb_to_grayscale(xg,num_output_channels=1)
            fake = self.decoder(xg,xg_gray,xt,M_a,M_t,xt,train=False)


            gen_mask = self.parsing(self.preprocess_parsing(fake))
            gen_mask = self.postprocess_parsing(gen_mask)
            gen_mask = gen_mask[0][0].cpu().numpy()
            mask_t = M_t[0][0].cpu().numpy()
            mask = np.zeros_like(gen_mask)
            for i in [1,2,3,4,5,6,7,8,9,10,11,12,13,17,18]:
                mask[gen_mask==i] = 1.0
                mask[mask_t==i] = 1.0

            self.mask = mask
        return fake

    def run_sr(self,input_np):
        input_np = cv2.cvtColor(input_np, cv2.COLOR_BGR2RGB)
        # prepare data
        input_np = input_np.transpose((2,0,1))
        input_np = np.array(input_np[np.newaxis, :])
        outputs_onnx = self.ort_session_sr.run(None, {'input_image':input_np.astype(np.uint8)})

        out_put_onnx = outputs_onnx[0]
        outimg = out_put_onnx[0,...].transpose(1,2,0)
        outimg = cv2.cvtColor(outimg, cv2.COLOR_BGR2RGB)
        return outimg


    def loadModel(self,align_path,blend_path,parsing_path):
        ckpt = torch.load(align_path, map_location=lambda storage, loc: storage)
        # self.netG.load_state_dict(ckpt['G'])
        self.netG.load_state_dict(ckpt['net_G_ema'])

        ckpt = torch.load(blend_path, map_location=lambda storage, loc: storage)
        self.decoder.load_state_dict(ckpt['G'],strict=False)

        self.parsing.load_state_dict(torch.load(parsing_path))


    def eval_model(self,*args):
        for arg in args:
            arg.eval()


In [ ]:

model = Infer(
            # 'checkpoint/Aligner/058-00008100.pth',
            'pretrained_models/epoch_00190_iteration_000400000_checkpoint.pt',
            'pretrained_models/Blender-401-00012900.pth',
            'pretrained_models/parsing.pth',
            'pretrained_models/epoch_20.pth',
            'pretrained_models/BFM')



AttributeError: _3D

In [ ]:
!pip install face_alignment

In [ ]:
import face_alignment

print(dir(face_alignment))

['FaceAlignment', 'LandmarksType', 'NetworkSize', '__author__', '__builtins__', '__cached__', '__doc__', '__email__', '__file__', '__loader__', '__name__', '__package__', '__path__', '__spec__', '__version__', 'api', 'detection', 'folder_data', 'utils']


In [ ]:
!nvidia-smi

Tue Feb 13 08:58:12 2024       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.104.05             Driver Version: 535.104.05   CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  Tesla T4                       Off | 00000000:00:04.0 Off |                    0 |
| N/A   51C    P0              27W /  70W |    205MiB / 15360MiB |      0%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+--

Upload Image

In [ ]:
from google.colab import files
import cv2
from google.colab.patches import cv2_imshow
import shutil
import os.path as osp


uploaded_imgs_path = 'assets'
if not osp.exists(uploaded_imgs_path) : os.makedirs(uploaded_imgs_path)

def upload_img():
  uploaded_img = files.upload()
  uploadded_img_name = list(uploaded_img.keys())[0]

  !mv "{uploadded_img_name}" "{uploaded_imgs_path}"

  print(f"move file {uploadded_img_name} to {uploaded_imgs_path} ")
  return os.path.join(uploaded_imgs_path,uploadded_img_name)
# print(uploaded_img.keys())
# for fn in uploaded_img.keys():
#   print('User uploaded file "{name}" with length {length} bytes'.format(
#       name=fn, length=len(uploaded_img[fn])))
up_mode = 0 #@param [0, 1]
if up_mode:
  src = upload_img()
  tgt = upload_img()
else:
  src = './assets/5.jpg'
  tgt = 'assets/fe54875c-2cf0-4147-b08a-80552a9f46be.jpg'

src_img = cv2.imread(src)
cv2_imshow(src_img)
tgt_img = cv2.imread(tgt)
cv2_imshow(tgt_img)


SyntaxError: invalid syntax (<ipython-input-23-85d1df7cce95>, line 1)